In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

/Users/Laura/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


Libraries imported.


In [2]:
address = 'Monaco, Monaco'

geolocator = Nominatim(user_agent="Ab_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Monaco, Monaco {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Monaco, Monaco 43.73844905, 7.424224092532953.


In [3]:
# create map of Matera using latitude and longitude values
map_matera = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.Marker(location=[latitude, longitude],popup="Monaco")
  
    
map_matera

In [4]:
CLIENT_ID = 'PBUYQF1I0ZUHP4AVQOX31OFTSSHQ0DGBQCLYAKU4HYHH4OX5' # your Foursquare ID
CLIENT_SECRET = 'YXOIHFLZNYLK53YTLR2POK5VYMRMZ3B1O1KK3AKVQSB0YLZR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PBUYQF1I0ZUHP4AVQOX31OFTSSHQ0DGBQCLYAKU4HYHH4OX5
CLIENT_SECRET:YXOIHFLZNYLK53YTLR2POK5VYMRMZ3B1O1KK3AKVQSB0YLZR


In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PBUYQF1I0ZUHP4AVQOX31OFTSSHQ0DGBQCLYAKU4HYHH4OX5&client_secret=YXOIHFLZNYLK53YTLR2POK5VYMRMZ3B1O1KK3AKVQSB0YLZR&v=20180605&ll=43.73844905,7.424224092532953&radius=500&limit=100'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6045013d5e57225b3b8151c8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Monte-Carlo',
  'headerFullLocation': 'Monte-Carlo, Monaco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 65,
  'suggestedBounds': {'ne': {'lat': 43.7429490545, 'lng': 7.43044081447308},
   'sw': {'lat': 43.7339490455, 'lng': 7.418007370592825}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc8f267762beee146bb3d38',
       'name': 'Hôtel Hermitage Monte-Carlo',
       'location': {'address': '1 square Beaumarchais',
        'lat': 43.73861751938308,
        'lng': 7.4260093134154,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43

In [7]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-8-4243e3e1ab9c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Hôtel Hermitage Monte-Carlo,Hotel,43.738618,7.426009
1,Chanel Boutique,Boutique,43.739505,7.425488
2,Thermes Marins Monte-Carlo,Spa,43.737498,7.426704
3,Louis Vuitton,Boutique,43.739061,7.426720
4,Crystal Bar,Bar,43.738031,7.425736
5,Hôtel de Paris,Hotel,43.739149,7.427685
6,Le Grill,Restaurant,43.738563,7.427572
7,Valentin,Italian Restaurant,43.739717,7.424846
8,Gucci,Shop & Service,43.738080,7.427231
9,Place du Casino,Plaza,43.739467,7.427746


In [9]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

65 venues were returned by Foursquare.


In [10]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))


There are 36 uniques categories.


In [11]:
# one hot encoding
monaco_onehot = pd.get_dummies(nearby_venues[['name']], prefix="", prefix_sep="")

# add categories column back to dataframe
monaco_onehot['categories'] = nearby_venues['categories'] 

#move categories column to the first position
categories = monaco_onehot['categories']
monaco_onehot.drop(labels=['categories'], axis=1,inplace = True)
monaco_onehot.insert(0, 'categories', categories)
monaco_onehot.head()

,categories,Adagio Palais Josephine,Auditorium Rainier III,Billionaire Club & Lounge,Boutique Orange,Buddha-Bar,Caffe Milano,Café de Paris,Carol Joy Spa at Fairmont Monte Carlo,Casino de Monte-Carlo,Casino du Café de Paris,Chanel Boutique,Christian Louboutin,Crystal Bar,Côté Jardin,Dior,Flashman's,Fnac,Formula 1 Grand Prix de Monaco,Gare SNCF de Monaco Monte-Carlo,Goyard,Gucci,Hermès,Horizon-Deck,Hotel Metropole Monte-Carlo,Hôtel Fairmont Monte Carlo,Hôtel Hermitage Monte-Carlo,Hôtel de Paris,Il Terrazzino,Indian Star,Jardins de la Petite Afrique,L'Escale,L'Horizon,La Boutique SBM,La Maison Du Caviar,La Marée Monaco,La Salle Empire,Ladurée,Le Bar Américain,Le Grill,Le Louis XV - Alain Ducasse,Le Métropole,Le Vistamar,Louis Vuitton,Marché U,Metcafé,Metropole ESPA,Miramar,Morelli's Gelato,New Indien Restaurant,Nike Store Monte Carlo,Novotel,Opéra de Monte-Carlo,Pacific,Place du Casino,Port Palace Hotel Monte Carlo,Rampoldi,Restaurant Joël Robuchon,Saphir Lounge & Bar,Sun Casino,Thermes Marins Monte-Carlo,Tip Top,Valentin,YOSHI,Yacht Club de Monaco,cooks
0,Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Boutique,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Spa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Boutique,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bar,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# let's group rows by categories and by taking the mean of the frequency of occurrence of each category

monaco_grouped = monaco_onehot.groupby('categories').mean().reset_index()
monaco_grouped.head()

,categories,Adagio Palais Josephine,Auditorium Rainier III,Billionaire Club & Lounge,Boutique Orange,Buddha-Bar,Caffe Milano,Café de Paris,Carol Joy Spa at Fairmont Monte Carlo,Casino de Monte-Carlo,Casino du Café de Paris,Chanel Boutique,Christian Louboutin,Crystal Bar,Côté Jardin,Dior,Flashman's,Fnac,Formula 1 Grand Prix de Monaco,Gare SNCF de Monaco Monte-Carlo,Goyard,Gucci,Hermès,Horizon-Deck,Hotel Metropole Monte-Carlo,Hôtel Fairmont Monte Carlo,Hôtel Hermitage Monte-Carlo,Hôtel de Paris,Il Terrazzino,Indian Star,Jardins de la Petite Afrique,L'Escale,L'Horizon,La Boutique SBM,La Maison Du Caviar,La Marée Monaco,La Salle Empire,Ladurée,Le Bar Américain,Le Grill,Le Louis XV - Alain Ducasse,Le Métropole,Le Vistamar,Louis Vuitton,Marché U,Metcafé,Metropole ESPA,Miramar,Morelli's Gelato,New Indien Restaurant,Nike Store Monte Carlo,Novotel,Opéra de Monte-Carlo,Pacific,Place du Casino,Port Palace Hotel Monte Carlo,Rampoldi,Restaurant Joël Robuchon,Saphir Lounge & Bar,Sun Casino,Thermes Marins Monte-Carlo,Tip Top,Valentin,YOSHI,Yacht Club de Monaco,cooks
0,Accessories Store,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Asian Restaurant,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Boutique,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Breakfast Spot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
num_top_venues = 1

for cat in monaco_grouped['categories']:
    print("----"+cat+"----")
    temp = monaco_grouped[monaco_grouped['categories']==cat].T.reset_index()
    temp.columns = ['name','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Accessories Store----
     name  freq
0  Goyard   1.0


----Asian Restaurant----
         name  freq
0  Buddha-Bar   1.0


----Bar----
               name  freq
0  Le Bar Américain   0.5


----Boutique----
            name  freq
0  Louis Vuitton  0.33


----Breakfast Spot----
        name  freq
0  L'Horizon   1.0


----Café----
            name  freq
0  Café de Paris   1.0


----Casino----
                      name  freq
0  Casino du Café de Paris  0.33


----Clothing Store----
     name  freq
0  Hermès   1.0


----Cocktail Bar----
         name  freq
0  Flashman's   1.0


----Convention Center----
                     name  freq
0  Auditorium Rainier III   1.0


----Department Store----
   name  freq
0  Fnac   1.0


----Dessert Shop----
      name  freq
0  Ladurée   0.5


----Electronics Store----
              name  freq
0  Boutique Orange   1.0


----French Restaurant----
                  name  freq
0  La Maison Du Caviar   0.2


----Grocery Store----
       name  freq
0  Marc

In [30]:
#Let us map the categories to colors

colors_array = cm.rainbow(np.linspace(0, 1, len(nearby_venues['categories'])))
rainbow = [colors.rgb2hex(i) for i in colors_array]
monaco_grouped['RGB']='red', 'blue', 'yellow','green','black','white','orange','purple','#2686f5','#1996f3','#10a2f0','#04b0ed','#B8860B','#8B4513','#B0C4DE','#9ACD32','#00FF7F','#7CFC00','#7B68EE','#663399','#F0E68C','#BDB76B', '#FFFFE0','#FF4500','#DB7093','#C71585','#FFC0CB','#FF1493','#6495ED','#DE3163','#9FE2BF','#CCCCFF','#DFFF00','#FFA07A','#CD5C5C','#F08080'
monaco_grouped

,categories,Adagio Palais Josephine,Auditorium Rainier III,Billionaire Club & Lounge,Boutique Orange,Buddha-Bar,Caffe Milano,Café de Paris,Carol Joy Spa at Fairmont Monte Carlo,Casino de Monte-Carlo,Casino du Café de Paris,Chanel Boutique,Christian Louboutin,Crystal Bar,Côté Jardin,Dior,Flashman's,Fnac,Formula 1 Grand Prix de Monaco,Gare SNCF de Monaco Monte-Carlo,Goyard,Gucci,Hermès,Horizon-Deck,Hotel Metropole Monte-Carlo,Hôtel Fairmont Monte Carlo,Hôtel Hermitage Monte-Carlo,Hôtel de Paris,Il Terrazzino,Indian Star,Jardins de la Petite Afrique,L'Escale,L'Horizon,La Boutique SBM,La Maison Du Caviar,La Marée Monaco,La Salle Empire,Ladurée,Le Bar Américain,Le Grill,Le Louis XV - Alain Ducasse,Le Métropole,Le Vistamar,Louis Vuitton,Marché U,Metcafé,Metropole ESPA,Miramar,Morelli's Gelato,New Indien Restaurant,Nike Store Monte Carlo,Novotel,Opéra de Monte-Carlo,Pacific,Place du Casino,Port Palace Hotel Monte Carlo,Rampoldi,Restaurant Joël Robuchon,Saphir Lounge & Bar,Sun Casino,Thermes Marins Monte-Carlo,Tip Top,Valentin,YOSHI,Yacht Club de Monaco,cooks,RGB
0,Accessories Store,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,red
1,Asian Restaurant,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,blue
2,Bar,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,yellow
3,Boutique,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,green
4,Breakfast Spot,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,black
5,Café,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,white
6,Casino,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.333333,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.333333,0.00,0.000000,0.0,0.0,0.0,0.0,orange
7,Clothing Store,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0

In [31]:
# create map
map_monaco = folium.Map(location=[latitude, longitude], zoom_start=16)

for lat, lon, poi, cluster, col in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories'],monaco_grouped['RGB']):
    label=folium.Popup(' Name: ' + str(poi) + ' Category: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=col,
        fill=True,
        fill_color=col,
        fill_opacity=0.8).add_to(map_monaco)
        
map_monaco.add_child(folium.LayerControl())
map_monaco

In [21]:
#Focusing on the Spa category
na = monaco_grouped[monaco_grouped['categories']=="Spa"]
na

,categories,Adagio Palais Josephine,Auditorium Rainier III,Billionaire Club & Lounge,Boutique Orange,Buddha-Bar,Caffe Milano,Café de Paris,Carol Joy Spa at Fairmont Monte Carlo,Casino de Monte-Carlo,Casino du Café de Paris,Chanel Boutique,Christian Louboutin,Crystal Bar,Côté Jardin,Dior,Flashman's,Fnac,Formula 1 Grand Prix de Monaco,Gare SNCF de Monaco Monte-Carlo,Goyard,Gucci,Hermès,Horizon-Deck,Hotel Metropole Monte-Carlo,Hôtel Fairmont Monte Carlo,Hôtel Hermitage Monte-Carlo,Hôtel de Paris,Il Terrazzino,Indian Star,Jardins de la Petite Afrique,L'Escale,L'Horizon,La Boutique SBM,La Maison Du Caviar,La Marée Monaco,La Salle Empire,Ladurée,Le Bar Américain,Le Grill,Le Louis XV - Alain Ducasse,Le Métropole,Le Vistamar,Louis Vuitton,Marché U,Metcafé,Metropole ESPA,Miramar,Morelli's Gelato,New Indien Restaurant,Nike Store Monte Carlo,Novotel,Opéra de Monte-Carlo,Pacific,Place du Casino,Port Palace Hotel Monte Carlo,Rampoldi,Restaurant Joël Robuchon,Saphir Lounge & Bar,Sun Casino,Thermes Marins Monte-Carlo,Tip Top,Valentin,YOSHI,Yacht Club de Monaco,cooks,RGB
32,Spa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,#62fbc4


In [23]:
#mapping the Spa map
map_na = folium.Map(location=[latitude, longitude], zoom_start=16)

for lat, lon, poi, cluster, col in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories'],na['RGB']):
    label=folium.Popup(' Name: ' + str(poi) + ' Category: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=col,
        fill=True,
        fill_color=col,
        fill_opacity=0.8).add_to(map_na)
        
map_na.add_child(folium.LayerControl())
map_na